In [ ]:
import torch
import torch.nn.functional as F
import torchvision.transforms as T
from transformers import AutoTokenizer, AutoModel, CLIPProcessor, CLIPModel
from PIL import Image
import numpy as np
import os
import pandas as pd

KeyboardInterrupt: 

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


Text Encoder/Image Encoder Stuff

In [ ]:
df = pd.read_csv("/content/EmptyColAnnotations - annotations.csv")
use_df = df[['image_id', 'class', 'x_min', 'y_min', 'x_max', 'y_max']]
use_df.head()

,image_id,class,x_min,y_min,x_max,y_max
0,1.jpg,pituitary,228.65402,266.70900,302.50067,370.87300
1,10.jpg,pituitary,213.46724,261.47446,275.70706,353.20157
2,10.jpg,meningioma,211.34334,259.60380,275.11280,352.85315
3,100.jpg,pituitary,263.05798,247.59110,388.75974,382.99933
4,1000.jpg,glioma,144.64809,123.53035,301.97308,241.46480


In [ ]:
pixels_per_cm=40
def calculate_tumor(x_min, y_min, x_max, y_max):
    width = x_max - x_min
    height = y_max - y_min
    avg_size_pixels = (width + height) / 2
    size_cm = round(avg_size_pixels / pixels_per_cm, 2)

    center_x = (x_min + x_max) / 2
    center_y = (y_min + y_max) / 2

    return center_x, center_y, size_cm


In [ ]:
def generate_manual_report(tumor_class, center_x, center_y, size_cm): #Templates generated by ChatGPT
    tumor_class = tumor_class.lower()

    templates = {
        "glioma": {
            "findings": (
                f"A heterogeneously enhancing mass is visualized in the vicinity of "
                f"the lesion center at approximately ({center_x}, {center_y}). "
                f"The lesion measures approximately {size_cm} cm in greatest dimension. "
                "It demonstrates an irregular shape with poorly defined margins. "
                "There is associated mild surrounding vasogenic edema, with subtle effacement of adjacent sulci. "
                "No evidence of midline shift or hydrocephalus is seen. No calcifications or hemorrhagic components identified."
            ),
            "impression": (
                "Imaging findings are consistent with a high-grade glioma. Recommend correlation with clinical presentation. "
                "Consider further evaluation with MRI perfusion and MR spectroscopy. Tissue sampling via stereotactic biopsy is advised."
            )
        },
        "meningioma": {
            "findings": (
                f"A well-circumscribed, homogeneously enhancing extra-axial mass is identified near "
                f"({center_x}, {center_y}), measuring approximately {size_cm} cm. "
                "The lesion is round in shape and causes mild mass effect without significant edema. "
                "No evidence of brain invasion or hemorrhage is noted."
            ),
            "impression": (
                "Findings are consistent with a meningioma. Surgical resection or monitoring can be considered based on clinical factors."
            )
        },
        "pituitary tumor": {
            "findings": (
                f"A lobulated lesion centered around ({center_x}, {center_y}), measuring approximately {size_cm} cm, "
                "located within the sellar region. The lesion demonstrates mild homogeneous enhancement. "
                "No surrounding edema or mass effect observed."
            ),
            "impression": (
                "Findings suggest a pituitary adenoma. Endocrine evaluation and neurosurgical consultation recommended."
            )
        }
    }

    # Fallback for unknown tumor class
    if tumor_class not in templates:
        findings = f"A lesion identified near ({center_x}, {center_y}) measuring approximately {size_cm} cm. Further characterization is needed."
        impression = "Tumor type not specified; recommend clinical correlation and further imaging."

    else:
        findings = templates[tumor_class]["findings"]
        impression = templates[tumor_class]["impression"]

    report = f"""Findings:
{findings}

Impression:
{impression}
"""
    return report



Findings:
A heterogeneously enhancing mass is visualized in the vicinity of the lesion center at approximately (200, 280). The lesion measures approximately 4.0 cm in greatest dimension. It demonstrates an irregular shape with poorly defined margins. There is associated mild surrounding vasogenic edema, with subtle effacement of adjacent sulci. No evidence of midline shift or hydrocephalus is seen. No calcifications or hemorrhagic components identified.

Impression:
Imaging findings are consistent with a high-grade glioma. Recommend correlation with clinical presentation. Consider further evaluation with MRI perfusion and MR spectroscopy. Tissue sampling via stereotactic biopsy is advised.



In [ ]:
descriptions = []

for idx, row in use_df.iterrows():
    center_x, center_y, size_cm = calculate_tumor(
        row['x_min'], row['x_max'], row['y_min'], row['y_max']
    )
    report = generate_manual_report(row['class'], center_x, center_y, size_cm)
    descriptions.append(report)

In [ ]:
for i in range(10):
  print(descriptions[i])

Findings:
A lesion identified near (247.68151, 336.686835) measuring approximately 1.33 cm. Further characterization is needed.

Impression:
Tumor type not specified; recommend clinical correlation and further imaging.

Findings:
A lesion identified near (237.47085, 314.454315) measuring approximately 1.57 cm. Further characterization is needed.

Impression:
Tumor type not specified; recommend clinical correlation and further imaging.

Findings:
A well-circumscribed, homogeneously enhancing extra-axial mass is identified near (235.47357, 313.982975), measuring approximately 1.58 cm. The lesion is round in shape and causes mild mass effect without significant edema. No evidence of brain invasion or hemorrhage is noted.

Impression:
Findings are consistent with a meningioma. Surgical resection or monitoring can be considered based on clinical factors.

Findings:
A lesion identified near (255.32454, 385.87953500000003) measuring approximately -0.27 cm. Further characterization is needed.


In [ ]:
df['description'] = descriptions
df.head()

,image_id,class,description,x_min,y_min,x_max,y_max
0,1.jpg,pituitary,Findings:\nA lesion identified near (247.68151...,228.65402,266.70900,302.50067,370.87300
1,10.jpg,pituitary,Findings:\nA lesion identified near (237.47085...,213.46724,261.47446,275.70706,353.20157
2,10.jpg,meningioma,"Findings:\nA well-circumscribed, homogeneously...",211.34334,259.60380,275.11280,352.85315
3,100.jpg,pituitary,Findings:\nA lesion identified near (255.32454...,263.05798,247.59110,388.75974,382.99933
4,1000.jpg,glioma,Findings:\nA heterogeneously enhancing mass is...,144.64809,123.53035,301.97308,241.46480


In [ ]:
from google.colab import files
csv_path = "/content/processed_reports.csv"
df.to_csv(csv_path, index=False)

In [ ]:
check_df = pd.read_csv("/content/processed_reports.csv")
check_df.head()

,image_id,class,description,x_min,y_min,x_max,y_max
0,1.jpg,pituitary,Findings:\nA lesion identified near (247.68151...,228.65402,266.70900,302.50067,370.87300
1,10.jpg,pituitary,Findings:\nA lesion identified near (237.47085...,213.46724,261.47446,275.70706,353.20157
2,10.jpg,meningioma,"Findings:\nA well-circumscribed, homogeneously...",211.34334,259.60380,275.11280,352.85315
3,100.jpg,pituitary,Findings:\nA lesion identified near (255.32454...,263.05798,247.59110,388.75974,382.99933
4,1000.jpg,glioma,Findings:\nA heterogeneously enhancing mass is...,144.64809,123.53035,301.97308,241.46480


In [ ]:
files.download(csv_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from google.colab import files

In [ ]:
df = pd.read_csv("/content/processed_reports.csv")
df.head()

,image_id,class,description,x_min,y_min,x_max,y_max
0,1.jpg,pituitary,Findings:\nA lesion identified near (247.68151...,228.65402,266.70900,302.50067,370.87300
1,10.jpg,pituitary,Findings:\nA lesion identified near (237.47085...,213.46724,261.47446,275.70706,353.20157
2,10.jpg,meningioma,"Findings:\nA well-circumscribed, homogeneously...",211.34334,259.60380,275.11280,352.85315
3,100.jpg,pituitary,Findings:\nA lesion identified near (255.32454...,263.05798,247.59110,388.75974,382.99933
4,1000.jpg,glioma,Findings:\nA heterogeneously enhancing mass is...,144.64809,123.53035,301.97308,241.46480


In [ ]:
df = df[['image_id', 'class', 'x_min', 'y_min', 'x_max', 'y_max', 'description']]
df.head()

,image_id,class,x_min,y_min,x_max,y_max,description
0,1.jpg,pituitary,228.65402,266.70900,302.50067,370.87300,Findings:\nA lesion identified near (247.68151...
1,10.jpg,pituitary,213.46724,261.47446,275.70706,353.20157,Findings:\nA lesion identified near (237.47085...
2,10.jpg,meningioma,211.34334,259.60380,275.11280,352.85315,"Findings:\nA well-circumscribed, homogeneously..."
3,100.jpg,pituitary,263.05798,247.59110,388.75974,382.99933,Findings:\nA lesion identified near (255.32454...
4,1000.jpg,glioma,144.64809,123.53035,301.97308,241.46480,Findings:\nA heterogeneously enhancing mass is...


In [ ]:
csv_path = "/content/finalized_reports.csv"
df.to_csv(csv_path, index=False)

In [ ]:
files.download(csv_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>